In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import os 

os.chdir('/Users/arnaud/Desktop/HEC/taxi-time/') 
from src.preprocessing import *
from src.master import master_preprocessing_X

In [2]:
Path_AirportData = "/Users/arnaud/Desktop/HEC/taxi-time/Data/Airport_Data.csv"
Path_WeatherData = "/Users/arnaud/Desktop/HEC/taxi-time/Data/weather_data_prep.csv"
Path_Aircraft = "/Users/arnaud/Desktop/HEC/taxi-time/Data/ACchar.xlsx"
Path_correspondance_Aircraft_Airport = "/Users/arnaud/Desktop/HEC/taxi-time/Correspondance.pkl"

In [3]:
master_df = master_preprocessing_X(Path_AirportData, Path_WeatherData, Path_Aircraft, Path_correspondance_Aircraft_Airport)


/Users/arnaud/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
missing_aircraft =  master_df[master_df['acType'].isna()]['flight'].unique().tolist()

In [28]:
df_flight_impute = (master_df[master_df['flight'].isin(missing_aircraft)]
 .groupby(['flight'])['acType']
 .value_counts()
 .rename("nb")
 .to_frame()
 .reset_index())

In [68]:
df_flight_impute.groupby(['flight'])[['nb']].max()

,nb
flight,
3281,31
3284,17
3287,40
3300,26
3315,30
...,...
5612,12
9198,7
9200,11


In [58]:
df_flight_impute.sort_values('nb', ascending=False).drop_duplicates(['flight','acType'])

,flight,acType,nb
100,4533,crj 100/200 standard,108
103,4574,crj 100/200 standard,103
164,5233,crj 900 standard,79
62,3592,crj 100/200 standard,70
198,5562,crj 700/701/702 standard,64
...,...,...,...
39,3432,crj 100/200 standard,1
69,3790,crj 100/200 standard,1
187,5496,crj 700/701/702 standard,1
182,5447,crj 900 standard,1


In [81]:
df_flight_impute.loc[df_flight_impute.groupby(['flight'])["nb"].idxmax()].drop('nb', axis = 1)

,flight,acType
0,3281,crj 100/200 standard
2,3284,crj 700/701/702 standard
5,3287,crj 100/200 standard
7,3300,crj 100/200 standard
9,3315,crj 900 standard
...,...,...
210,5612,crj 700/701/702 standard
211,9198,crj 100/200 standard
214,9200,crj 900 standard
216,9202,crj 900 standard


In [99]:
def impute_acType(df):
    """
    function imputing the acType to missing acType using most often used aircraft on the flight number
    
    input: original pd.dataFrame with missing acType
    output: pd.DataFrame with no missing acType
    """
    missing_aircraft =  df[df['acType'].isna()]['flight'].unique().tolist()
    
    df_flight_impute = (df[df['flight'].isin(missing_aircraft)]
                                        .groupby(['flight'])['acType']
                                        .value_counts()
                                        .rename("nb")
                                        .to_frame()
                                        .reset_index())
    
    df_flight_impute = df_flight_impute.loc[df_flight_impute.groupby(['flight'])["nb"].idxmax()].drop('nb', axis = 1)
    df_final = df[(df['acType'].isna()) & (~df['flight'].isna())].drop('acType', axis = 1).merge(df_flight_impute, how = 'left', on = ["flight"])

    return df_final

In [96]:
impute = df_flight_impute.loc[df_flight_impute.groupby(['flight'])["nb"].idxmax()].drop('nb', axis = 1)   

In [98]:
master_df[(master_df['acType'].isna()) & (~master_df['flight'].isna())].drop('acType', axis = 1).merge(impute, how = 'left', on = ["flight"])

,date,carrier,flight,sto,runway,stand,aldt,eibt,cibt,aibt,...,MTOW,Max Ramp\nMax Taxi,Main Gear Config,ICAO Code,Wake Category,ATCT Weight Class,Years Manufactured,Note,"Parking Area (WS x Length), sf",acType
0,2018-07-30,CAR2,5496,7/30/2018 18:04,RUNWAY05,STAND112,7/30/2018 19:08,7/30/2018 19:15,7/30/2018 19:17,7/30/2018 19:18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,crj 900 standard
1,2018-07-30,CAR3,4873,7/30/2018 18:14,RUNWAY07,STAND121,7/30/2018 19:13,7/30/2018 19:20,7/30/2018 19:27,7/30/2018 19:27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,crj 100/200 standard
2,2018-07-31,CAR4,5090,7/31/2018 18:55,RUNWAY09,STAND123,7/31/2018 22:26,7/31/2018 22:32,7/31/2018 22:33,7/31/2018 22:33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,crj 100/200 standard
3,2018-07-31,CAR4,5057,7/31/2018 20:12,RUNWAY10,STAND25,7/31/2018 22:39,7/31/2018 22:43,7/31/2018 22:46,7/31/2018 22:47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,crj 100/200 standard
4,2018-07-31,CAR2,5526,7/31/2018 19:03,RUNWAY09,STAND99,7/31/2018 22:45,7/31/2018 22:53,7/31/2018 22:53,7/31/2018 22:55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,crj 900 standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,2019-01-12,CAR4,5078,1/12/2019 13:39,RUNWAY06,STAND100,1/12/2019 19:43,1/12/2019 19:51,1/12/2019 14:16,1/12/2019 19:48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,crj 900 standard
190,2019-01-13,CAR4,5015,1/13/2019 18:17,RUNWAY08,STAND151,1/13/2019 19:08,1/13/2019 19:16,1/13/2019 19:17,1/13/2019 19:18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,crj 100/200 standard
191,2019-01-14,CAR4,5370,1/14/2019 14:23,RUNWAY09,STAND124,1/14/2019 14:48,1/14/2019 14:56,1/14/2019 14:58,1/14/2019 14:58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,crj 100/200 standard
192,2019-01-14,CAR3,4042,1/14/2019 15:19,RUNWAY09,STAND152,1/14/2019 16:37,1/14/2019 16:43,1/14/2019 16:14,1/14/2019 16:43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,crj 100/200 standard


,date,carrier,flight,sto,runway,stand,aldt,eibt,cibt,aibt,...,MTOW,Max Ramp\nMax Taxi,Main Gear Config,ICAO Code,Wake Category,ATCT Weight Class,Years Manufactured,Note,"Parking Area (WS x Length), sf",acType
0,2018-07-30,CAR2,5496,7/30/2018 18:04,RUNWAY05,STAND112,7/30/2018 19:08,7/30/2018 19:15,7/30/2018 19:17,7/30/2018 19:18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,crj 900 standard
1,2018-07-30,CAR3,4873,7/30/2018 18:14,RUNWAY07,STAND121,7/30/2018 19:13,7/30/2018 19:20,7/30/2018 19:27,7/30/2018 19:27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,crj 100/200 standard
2,2018-07-31,CAR4,5090,7/31/2018 18:55,RUNWAY09,STAND123,7/31/2018 22:26,7/31/2018 22:32,7/31/2018 22:33,7/31/2018 22:33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,crj 100/200 standard
3,2018-07-31,CAR4,5057,7/31/2018 20:12,RUNWAY10,STAND25,7/31/2018 22:39,7/31/2018 22:43,7/31/2018 22:46,7/31/2018 22:47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,crj 100/200 standard
4,2018-07-31,CAR2,5526,7/31/2018 19:03,RUNWAY09,STAND99,7/31/2018 22:45,7/31/2018 22:53,7/31/2018 22:53,7/31/2018 22:55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,crj 900 standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,2019-01-12,CAR4,5078,1/12/2019 13:39,RUNWAY06,STAND100,1/12/2019 19:43,1/12/2019 19:51,1/12/2019 14:16,1/12/2019 19:48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,crj 900 standard
190,2019-01-13,CAR4,5015,1/13/2019 18:17,RUNWAY08,STAND151,1/13/2019 19:08,1/13/2019 19:16,1/13/2019 19:17,1/13/2019 19:18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,crj 100/200 standard
191,2019-01-14,CAR4,5370,1/14/2019 14:23,RUNWAY09,STAND124,1/14/2019 14:48,1/14/2019 14:56,1/14/2019 14:58,1/14/2019 14:58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,crj 100/200 standard
192,2019-01-14,CAR3,4042,1/14/2019 15:19,RUNWAY09,STAND152,1/14/2019 16:37,1/14/2019 16:43,1/14/2019 16:14,1/14/2019 16:43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,crj 100/200 standard
